<a href="https://colab.research.google.com/github/HyeonhoonLee/KoNLP/blob/master/GPT/tf_KoGPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Korean Language Modeling using GPT2

- The contents of this notebook is modified from [NLP-Kr](https://github.com/NLP-kr/tensorflow-ml-nlp-tf2)
- The source codes are based on [HuggingFace Transformers](https://github.com/huggingface/transformers) and [SKT KoGPT2](https://github.com/SKT-AI/KoGPT2)
- To understand the pricinpale of GPT2, see this kindful page of [illustrated GPT-2](https://jalammar.github.io/illustrated-gpt2/).
- The codes below use Tensorflow 2.3.0

##Libraries and modules

In [ ]:
!pip install gluonnlp
!pip install transformers

In [ ]:
!pip install mxnet

In [6]:
import os

import numpy as np
import tensorflow as tf

In [5]:
# To use the word vocab API (https://nlp.gluon.ai/api/vocab.html).
import gluonnlp as nlp  

# Do not use the tokenizer of Transformers. KoGPT2 model used this type of Tokenizer.
# This tokenizer supports subword tokenization such as BPE(Byte pair Encoding).
# (https://nlp.gluon.ai/api/data.html)
from gluonnlp.data import SentencepieceTokenizer

# To use Open AI GPT2 Model transformer with a language modeling head on top (linear layer with weights tied to the input embeddings).
# (https://huggingface.co/transformers/model_doc/gpt2.html#gpt2lmheadmodel)
from transformers import TFGPT2LMHeadModel

##Korean GPT2 Model

In [7]:
!wget https://www.dropbox.com/s/nzfa9xpzm4edp6o/gpt_ckpt.zip -O gpt_ckpt.zip
!unzip -o gpt_ckpt.zip

--2020-11-03 06:43:36--  https://www.dropbox.com/s/nzfa9xpzm4edp6o/gpt_ckpt.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.1, 2620:100:6018:1::a27d:301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/nzfa9xpzm4edp6o/gpt_ckpt.zip [following]
--2020-11-03 06:43:36--  https://www.dropbox.com/s/raw/nzfa9xpzm4edp6o/gpt_ckpt.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2e80bd458e0ccc36f5f0ada2d9.dl.dropboxusercontent.com/cd/0/inline/BCdKDmTgEm3f-XkOreGeGt1oBrZt0rzfMZ0n_wU11LD246ByaqhXK1sPiLdoj_9xoxq7jcpWVWsErdpZGUGeTcs3p5vJ59stFMvMKTydBZayo25KS7xbIWXfQPZty34_xeo/file# [following]
--2020-11-03 06:43:36--  https://uc2e80bd458e0ccc36f5f0ada2d9.dl.dropboxusercontent.com/cd/0/inline/BCdKDmTgEm3f-XkOreGeGt1oBrZt0rzfMZ0n_wU11LD246ByaqhXK1sPiLdoj_9xoxq7jcpWVWsErdpZGUGeTcs3p5vJ59stFMvMKTydBZayo25KS7

In [8]:
# To define the GPT2Model class.
class GPT2Model(tf.keras.Model):
  def __init__(self, dir_path):
    super(GPT2Model, self).__init__()
    self.gpt2 = TFGPT2LMHeadModel.from_pretrained(dir_path)
  
  # object gpt2 includes 4 outputs of tuple (last_hidden_states, past, hidden_state, attentions)
  def call(self, inputs):
    return self.gpt2(inputs)[0]

In [12]:
# We used Colab.
BASE_MODEL_PATH = '/content/drive/My Drive/ModelCollection/gpt_ckpt/'
# Create pre-trained gpt2 model.
gpt_model = GPT2Model(BASE_MODEL_PATH)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/drive/My Drive/ModelCollection/gpt_ckpt/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


## Tokenizer

In [13]:
TOKENIZER_PATH = os.path.join(BASE_MODEL_PATH + 'gpt2_kor_tokenizer.spiece')

# To make tokenizer for KoGPT2.
tokenizer = SentencepieceTokenizer(TOKENIZER_PATH)

# To define the word dictionary.
vocab = nlp.vocab.BERTVocab.from_sentencepiece(TOKENIZER_PATH,
                                               mask_token=None,
                                               sep_token=None,
                                               cls_token=None,
                                               unknown_token='<unk>',
                                               padding_token='<pad>',
                                               bos_token='<s>',
                                               eos_token='</s>')

## Sentence Generator

In [14]:
def tf_top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-99999):
    _logits = logits.numpy()
    top_k = min(top_k, logits.shape[-1])  
    
    # top-k sampling method
    if top_k > 0:
        indices_to_remove = logits < tf.math.top_k(logits, top_k)[0][..., -1, None]
        _logits[indices_to_remove] = filter_value
    
    # Nuclus sampling method
    if top_p > 0.0:
        sorted_logits = tf.sort(logits, direction='DESCENDING')
        sorted_indices = tf.argsort(logits, direction='DESCENDING')
        cumulative_probs = tf.math.cumsum(tf.nn.softmax(sorted_logits, axis=-1), axis=-1)

        sorted_indices_to_remove = cumulative_probs > top_p
        sorted_indices_to_remove = tf.concat([[False], sorted_indices_to_remove[..., :-1]], axis=0)
        indices_to_remove = sorted_indices[sorted_indices_to_remove].numpy().tolist()
        
        _logits[indices_to_remove] = filter_value
    return tf.constant([_logits])

In [15]:
def generate_sent(seed_word, model, max_step=100, greedy=False, top_k=0, top_p=0.):
    sent = seed_word   # Input sentence or word
    toked = tokenizer(sent)  # Tokenizing
    
    for _ in range(max_step):  # max_step is the maximum size of sentence generating
        input_ids = tf.constant([vocab[vocab.bos_token],]  + vocab[toked])[None, :] 
        outputs = model(input_ids)[:, -1, :] # Output is the last subword of sentence.
        if greedy:  # greedy search with Maximum Likelihood Estimation.
            gen = vocab.to_tokens(tf.argmax(outputs, axis=-1).numpy().tolist()[0])
        else:    # Using top_k & Nucleus sampling.
            output_logit = tf_top_k_top_p_filtering(outputs[0], top_k=top_k, top_p=top_p)
            gen = vocab.to_tokens(tf.random.categorical(output_logit, 1).numpy().tolist()[0])[0]
        if gen == '</s>': # Stop generating when meeting this special token.
            break
        sent += gen.replace('▁', ' ')
        toked = tokenizer(sent)

    return sent